# Submission Information:

### Team Member 1:
* UNI:  ww2439
* Name: Weiwei Wang

### Team Member 2 :
* UNI: zs2319 
* Name: Zhengyuan Shi

# Step0 - Import Libraries, Load Data [0 points]

This is the basic step where you can load the data and create train and test sets for internal validation as per your convinience.

In [1]:
#write code below, you can make multiple cells
import numpy  as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')



# Step1 - Exploration and Preparation [10 points]

In this step, we expect you to look into the data and try to understand it before modeling. This understanding may lead to some basic data preparation steps which are common across the two model sets required.

In [2]:
#write code below, you can make multiple cells

# Fetch Data 
data_raw = pd.read_csv("/Users/weiweiwang/Desktop/AML1/HW3/data.csv")
#data_holdout_raw = pd.read_csv("/Users/weiweiwang/Desktop/AML1/HW3/holdout.csv")

data_holdout_raw = pd.read_csv("homework-iii-WeiweiVivianWang/data/holdout.csv")
# Split Data into X and y
X = data_raw.drop('subscribed', axis=1)
y = data_raw['subscribed']
y.replace(to_replace = "yes", value = 1, inplace = True)
y.replace(to_replace = "no", value = 0, inplace = True)

# Define Parameters 
random_state = 7
test_size = 0.25

# Split Data into Random Training and Test Subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                    random_state=random_state)

#holdout

X_holdout = data_holdout_raw.drop('ID', axis=1)


In [3]:
# Recode Missing Values 
def recode_NA(X):
    """Returns X with missing values coded as NaNs"""
    #Index of Features where NA = "unknown"
    j = []
    j.extend(range(1, 7))
    
    for i in j:
        X.iloc[:, i].replace(to_replace="unknown", value=np.nan, inplace=True)
     
    #Feature where NA = "nonexistent"
    X.iloc[:, 14].replace(to_replace="nonexistent", value=np.nan, inplace=True)
    
    return X

In [4]:
#write code below, you can make multiple cells
##feature selection
def select_features(X):
    """Returns a subset of X
    
    Remove features that are irrelavent.
    """
    
    # Remove Selected Features
    X.drop(X.columns[10], axis=1, inplace=True)
    
    
    return X

In [5]:
# One-Hot-Encoding get dummy variables
def encode_OH(X):
    """Returns X with categorical features encoded as one-hot"""

    #List of column names for continuous variables
    list_con = ["age","campaign","prev_days","prev_contacts","emp_var_rate","cons_price_idx","cons_conf_idx",
               "euribor3m","nr_employed"] 
    
    #List of column names for categorical variables
    list_cat = ['job', 'marital_status', 'education', 'credit_default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'prev_outcomes']
    
    return pd.get_dummies(X, columns = list_cat)

In [6]:
X_train = recode_NA(X_train)
X_train = select_features(X_train)
X_train = encode_OH(X_train)
X_test = recode_NA(X_test)
X_test = select_features(X_test)
X_test = encode_OH(X_test)

X_holdout = recode_NA(X_holdout)
X_holdout = select_features(X_holdout)
X_holdout = encode_OH(X_holdout)
X_holdout.insert(31, 'credit_default_yes', 0)


X_whole = recode_NA(X)
X_whole = select_features(X)
X_whole = encode_OH(X)
y_whole = y

# Step2 - ModelSet1 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set1:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. Any classification algorithm covered in class apart from tree-based models can be tested here.

In [7]:
# Define Parameters
random_state = 0
test_size = 0.25
cv = ShuffleSplit(n_splits=10, test_size=test_size, random_state=random_state)


In [8]:
#Logistic Regression
from sklearn.linear_model import LogisticRegressionCV

logistic = LogisticRegressionCV(class_weight='balanced')
#logistic = logistic.fit(X_train,y_train)
logistic = logistic.fit(X_train,y_train)

logistic.predict(X_test)


result_holdout = logistic.predict_proba(X_test)[:,1]
result_test_predict = logistic.predict_proba(X_test)[:,1]
roc_auc_score(y_test, result_test_predict)




0.78362432208736321

In [9]:
test_hold = pd.read_csv('holdout.csv')
test_hold['subscribed'] = result_holdout
test_hold.to_csv('sub_holdout.csv', index=False)

In [12]:
###SVM
from sklearn.svm import SVC 
from sklearn.metrics import roc_auc_score
model_svm=make_pipeline(StandardScaler(), SVC(probability=True))
model_svm.fit(X_train,y_train)
svm_prob=model_svm.predict_proba(X_test)
roc_auc_score(y_test,svm_prob[:,1])

0.71146996347808966

In [11]:
svm_result_holdout = model_svm.predict_proba(X_holdout)[:,1]
#svm_result_test_predict = model_svm.predict_proba(X_test)[:,1]

test_hold = pd.read_csv('holdout.csv')
test_hold['subscribed'] = svm_result_holdout
test_hold.to_csv('sub_holdout.csv', index=False)

In [10]:
###QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
QDA = make_pipeline(StandardScaler(), QuadraticDiscriminantAnalysis())
#QDA.fit(X_train, y_train)
QDA.fit(X_whole, y_whole)
QDA.predict(X_test)
QDA_prob = QDA.predict_proba(X_test)
roc_auc_score(y_test,QDA_prob[:,1])

0.74480402030409842

In [11]:
##Nerual Network
from sklearn.neural_network import MLPClassifier

MLP = make_pipeline(StandardScaler(), MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1))
MLP.fit(X_whole, y_whole)
MLP.predict(X_test)
MLP_prob = MLP.predict_proba(X_test)
nn_score = roc_auc_score(y_test,MLP_prob[:,1])
print (nn_score)




0.818061271148


In [12]:
nn_result_holdout = MLP.predict_proba(X_holdout)[:,1]
#nn_result_test_predict = MLP.predict_proba(X_test)[:,1]

test_hold = pd.read_csv('holdout.csv')
test_hold['subscribed'] = nn_result_holdout
test_hold.to_csv('sub_holdout.csv', index=False)

In [13]:
###LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = make_pipeline(StandardScaler(), LinearDiscriminantAnalysis())
LDA.fit(X_whole, y_whole)
LDA.predict(X_test)
LDA_prob = LDA.predict_proba(X_test)
roc_auc_score(y_test,LDA_prob[:,1])

0.78698277508638503

In [14]:
lda_result_holdout = LDA.predict_proba(X_holdout)[:,1]
#nn_result_test_predict = MLP.predict_proba(X_test)[:,1]

test_hold = pd.read_csv('holdout.csv')
test_hold['subscribed'] = lda_result_holdout
test_hold.to_csv('sub_holdout.csv', index=False)

# Step3 - ModelSet2 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set2:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. We encourage you to try decition tree, random forest and gradient boosted tree methods here and pick the one which you think works best.

In [15]:
#write code below, you can make multiple cells

##Random forest without grid search
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.predict(X_test)
rf_prob = rf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test,rf_prob)

0.75102366978636703

In [17]:
###Random forest with grid search

from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
# Build a classification task using 3 informative features
#X, y = make_classification(n_samples=1000,
#                           n_features=10,
#                           n_informative=3,
#                           n_redundant=0,
#                           n_repeated=0,
#                           n_classes=2,
#                           random_state=0,
#                           shuffle=False)


rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True) 

param_grid = { 
    'n_estimators': [100, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
print (CV_rfc.best_params_)

{'max_features': 'sqrt', 'n_estimators': 700}


In [18]:
rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=100, oob_score = True) 
rfc.fit(X_train, y_train)
rfc.predict(X_test)
rfc_prob = rfc.predict_proba(X_test)[:, 1]
roc_auc_score(y_test,rfc_prob)


0.77626017954194237

In [16]:
###XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb.predict(X_test)
xgb_prob = xgb.predict_proba(X_test)
roc_auc_score(y_test,xgb_prob[:,1])

0.78977350338305174

In [17]:
xgb = XGBClassifier()
xgb.fit(X_whole, y_whole)
xgb.predict(X_holdout)
xgb_prob = xgb.predict_proba(X_holdout)
roc_auc_score(y_test,xgb_prob[:,1])

xgb_result_holdout = xgb.predict_proba(X_holdout)[:,1]
xgb_result_test_predict = xgb.predict_proba(X_test)[:,1]

test_hold = pd.read_csv('holdout.csv')
test_hold['subscribed'] = xgb_result_holdout
test_hold.to_csv('sub_holdout.csv', index=False)

In [18]:
###AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=200)
ada.fit(X_train, y_train)
ada.predict(X_test)
ada_prob = ada.predict_proba(X_test)[:,1]
roc_auc_score(y_test, ada_prob)


0.78570052901784426

In [19]:
#ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier
ext = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)
ext.fit(X_train, y_train)
ext.predict(X_test)
ext_prob = ext.predict_proba(X_test)
roc_auc_score(y_test, ext_prob[:,1])

0.768575819138407

In [22]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(max_depth = 8, 
                                n_estimators = 120, 
                                learning_rate = 0.05, 
                                max_features = 'sqrt', 
                                min_samples_split = 500, 
                                min_samples_leaf=50)



gb.fit(X_train, y_train)
gb.predict(X_test)
gb_prob = gb.predict_proba(X_test)
roc_auc_score(y_test, gb_prob[:,1])

0.79405166027099139

In [23]:
gb = GradientBoostingClassifier(max_depth = 8, 
                                n_estimators = 120, 
                                learning_rate = 0.05, 
                                max_features = 'sqrt', 
                                min_samples_split = 500, 
                                min_samples_leaf=50)
gb.fit(X_whole, y_whole)
gb.predict(X_holdout)
gb_prob = gb.predict_proba(X_holdout)[:,1]

test_hold = pd.read_csv('holdout.csv')
test_hold['subscribed'] = gb_prob
test_hold.to_csv('sub_holdout.csv', index=False)


# Step4 - Ensemble [20 points + 10 Bonus points]

In this step, we expect you to use the models created before and create new predictions. You should definitely try poor man's stacking but we encourage you to think of different ensemble techniques as well. We will judge your creativity and improvement in model performance using ensemble models and you can potentially earn 10 bonus points here.

In [23]:
##Model averaging--voting classifier

from sklearn.ensemble import VotingClassifier
voting1 = VotingClassifier([('Gradient Boosting',gb),
                           ('Neural Network',MLP),
                          ('LDA',LDA)],voting = 'soft')

voting1.fit(X_train,y_train)
voting1_prob = voting1.predict_proba(X_test)
roc_auc_score(y_test, voting1_prob[:,1])

0.79117541514009715

In [24]:
#Geting prediction probability for holdout set.

voting1.fit(X_whole, y_whole)
voting1.predict(X_holdout)
voting1_prob = voting1.predict_proba(X_holdout)[:,1]

test_hold = pd.read_csv('holdout.csv')
test_hold['subscribed'] = voting1_prob
test_hold.to_csv('sub_holdout.csv', index=False)

In [32]:
from mlxtend.classifier import StackingClassifier

sclf = StackingClassifier(classifiers=[gb,
                                       MLP,
                                       LDA],
                          meta_classifier=MLP)


sclf.fit(X_train,y_train)

sclf_prob = sclf.predict_proba(X_test)
roc_auc_score(y_test, sclf_prob[:,1])

0.6790488097098506

In [20]:
##Creative method


In [ ]:
#Resampling Techniques

from imblearn.under_sampling import RandomUnderSampler 

from imblearn.pipeline import make_pipeline as make_imb_pipeline

rus = RandomUnderSampler(replacement = False)
X_train_subsample, y_train_subsample = rus.fit_sample(X_train, y_train)

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_train_oversample, y_train_oversample = ros.fit_sample(X_train,y_train)
resample_pipe = make_imb_pipeline(RandomOverSampler(), GradientBoostingClassifier(random_state = 22))
param_grid = {'gradientboostingclassifier__n_estimators':[100,200],
              'gradientboostingclassifier__max_depth':[2,4,6,8,10],
              'gradientboostingclassifier__max_features':['auto','sqrt','log2']}

resamp_model = GridSearchCV(estimator = resample_pipe,
                           param_grid = param_grid,
                           cv = 5, 
                           refit = True)
resamp_model.fit(X_train, y_train)
resamp_prob = resamp_model.predict_proba(X_test)[:,1]
RAscore_resamp = roc_auc_score(y_test, resamp_prob)
print(RAscore_resamp)

In [ ]:
resamp_model.fit(X_whole, y_whole)
resamp_prob = resamp_model.predict_proba(X_holdout)[:,1]

test_hold = pd.read_csv('holdout.csv')
test_hold['subscribed'] = resamp_prob
test_hold.to_csv('sub_holdout.csv', index=False)


In [39]:
RAscore_resamp
print (resamp_model.best_params_)

{'gradientboostingclassifier__max_depth': 2, 'gradientboostingclassifier__max_features': 'auto', 'gradientboostingclassifier__n_estimators': 268}


In [39]:
#write code below, you can make multiple cells
##Since the max score comes from neural network, we assert that roc_auc score >= 0.78.
assert nn_score >= 0.80